In [2]:
import json
import math
import pickle
import sys
import threading
import time
import traceback
from collections import defaultdict
from concurrent.futures.thread import ThreadPoolExecutor
import numpy as np
from elote import EloCompetitor
from katrain.core.constants import (
    AI_LOCAL,
    AI_RANK,
    AI_TENUKI,
    AI_WEIGHTED,
    AI_PICK,
    AI_TERRITORY,
    AI_POLICY,AI_INFLUENCE
)

from settings import Logger
import matplotlib.pyplot as plt

[INFO   ] [Logger      ] Record log in C:\Users\sande\.kivy\logs\kivy_20-07-06_10.txt
[INFO   ] [deps        ] Successfully imported "kivy_deps.gstreamer" 0.1.17
[INFO   ] [deps        ] Successfully imported "kivy_deps.angle" 0.2.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.glew" 0.2.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.sdl2" 0.2.0
[INFO   ] [Kivy        ] v2.0.0rc2, git-78fb93b, 20200429
[INFO   ] [Kivy        ] Installed at "C:\Users\sande\Anaconda3\lib\site-packages\kivy\__init__.py"
[INFO   ] [Python      ] v3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)]
[INFO   ] [Python      ] Interpreter at "C:\Users\sande\Anaconda3\python.exe"


In [3]:
DB_FILENAME = "tournament_ai_performance.pickle"
class AI:
    pass

class FixedEloCompetitor(EloCompetitor):  # rating doesn't update on wins/losses
    pass
    
with open(DB_FILENAME, "rb") as f:
    ai_database,_ = pickle.load(f)

In [18]:
from sklearn.linear_model import LinearRegression
ranked = [ai for ai in ai_database if ai.strategy==AI_RANK]
calibrate = [ai for ai in ai_database if ai.strategy==AI_WEIGHTED]
policy = [ai for ai in ai_database if ai.strategy==AI_POLICY and ai.ai_settings['opening_moves']==0][0]
policy_elo = 1700
kyu_elo = [(r.ai_settings['kyu_rank'],r.elo_comp.rating) for r in ranked]
#kyu_elo.append( (-5,policy.elo_comp.rating) ) # policy ~5d
kyu_elo = sorted(kyu_elo)
kyu,elo = zip(*kyu_elo)


In [ ]:
kyu2 = np.array(kyu)[:,np.newaxis]
reg = LinearRegression().fit(kyu2,elo)
plt.plot(kyu,elo,kyu,reg.predict(kyu2))

In [21]:
list(zip(kyu,reg.predict(kyu2)))

[(-4, 1392.5226294653357),
 (-3, 1328.2407152976634),
 (-2, 1263.9588011299913),
 (-1, 1199.6768869623193),
 (0, 1135.3949727946472),
 (1, 1071.113058626975),
 (2, 1006.8311444593029),
 (3, 942.5492302916308),
 (4, 878.2673161239586),
 (5, 813.9854019562865),
 (6, 749.7034877886144),
 (7, 685.4215736209424),
 (8, 621.1396594532702),
 (9, 556.8577452855981),
 (10, 492.5758311179259),
 (11, 428.2939169502538),
 (12, 364.0120027825817),
 (13, 299.7300886149095),
 (14, 235.44817444723742),
 (15, 171.16626027956522),
 (16, 106.88434611189314),
 (17, 42.60243194422105),
 (18, -21.679482223451032)]

In [6]:
wt_elo =  [([r.ai_settings['weaken_fac']],r.elo_comp.rating) for r in calibrate]
x,y = zip(*wt_elo)
reg = LinearRegression().fit(x, y)
py = reg.predict(x)

In [7]:
DB_FILENAME = "calibrated_ai_performance.scoreloss.pickle"
with open(DB_FILENAME, "rb") as f:
    ai_database_sl,_ = pickle.load(f)
str_elo =  [([r.ai_settings['strength']],r.elo_comp.rating) for r in ai_database_sl]
[(a[0],round(b)) for a,b in str_elo]

[(0.0, 539),
 (0.05, 625),
 (0.1, 859),
 (0.2, 1035),
 (0.3, 1201),
 (0.4, 1299),
 (0.5, 1346),
 (0.75, 1374),
 (1.0, 1386),
 (1.5, 1376),
 (2.0, 1370),
 (3.0, 1383)]

In [8]:
DB_FILENAME = "calibrated_ai_performance.jank.pickle"
with open(DB_FILENAME, "rb") as f:
    ai_database_jank,_ = pickle.load(f)

In [15]:
for stratname in ['AI_LOCAL','AI_TENUKI','AI_TERRITORY','AI_INFLUENCE','AI_PICK']:
    strat = eval(stratname)
    pick_ais = [ai for ai in ai_database_jank if ai.strategy==strat]
    settings_elo =  [([r.ai_settings['pick_frac'],r.ai_settings['pick_n']],min(policy_elo,r.elo_comp.rating)) for r in pick_ais]
    settings,elo = zip(*settings_elo)
    frac, n = zip(*settings)
    
    frac_x = np.unique(frac)
    n_y = np.unique(n)
    matrix = np.zeros( (len(n_y),len(frac_x)) )    
    
    for (f,n),elo in settings_elo:
        xi, = np.where(frac_x==f)
        yi, = np.where(n_y==n)
        matrix[yi[0]][xi[0]] = round(elo)
        
    print(f"{stratname}_ELO_GRID = [{repr(list(frac_x))},{repr(list(n_y))},{repr(list(list(x) for x in matrix))}]")

AI_LOCAL_ELO_GRID = [[0.0, 0.05, 0.1, 0.2, 0.3, 0.5, 0.75, 1.0],[0, 5, 10, 15, 25, 50],[[450.0, 704.0, 1190.0, 1282.0, 1520.0, 1464.0, 1617.0, 1700.0], [153.0, 1079.0, 1144.0, 1335.0, 1440.0, 1525.0, 1700.0, 1700.0], [606.0, 1185.0, 1268.0, 1484.0, 1404.0, 1544.0, 1700.0, 1700.0], [911.0, 1230.0, 1312.0, 1352.0, 1432.0, 1564.0, 1700.0, 1700.0], [1212.0, 1326.0, 1393.0, 1441.0, 1446.0, 1601.0, 1700.0, 1700.0], [1321.0, 1395.0, 1399.0, 1473.0, 1582.0, 1700.0, 1700.0, 1700.0]]]
AI_TENUKI_ELO_GRID = [[0.0, 0.05, 0.1, 0.2, 0.3, 0.5, 0.75, 1.0],[0, 5, 10, 15, 25, 50],[[491.0, 394.0, 527.0, 676.0, 732.0, 959.0, 1415.0, 1700.0], [21.0, 456.0, 620.0, 719.0, 812.0, 1027.0, 1308.0, 1700.0], [306.0, 480.0, 689.0, 705.0, 925.0, 1154.0, 1397.0, 1700.0], [286.0, 534.0, 636.0, 796.0, 968.0, 1077.0, 1443.0, 1700.0], [534.0, 645.0, 665.0, 846.0, 999.0, 1134.0, 1422.0, 1700.0], [746.0, 719.0, 806.0, 975.0, 1048.0, 1370.0, 1505.0, 1700.0]]]
AI_TERRITORY_ELO_GRID = [[0.0, 0.05, 0.1, 0.2, 0.3, 0.5, 0.75, 1.